<a href="https://colab.research.google.com/github/dinggga/super-broccoli/blob/main/220314ss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1번

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [2]:
X,y = mnist['data'], mnist['target']
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [3]:
# knn
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)

knn_clf.predict(X_test)

array(['7', '2', '1', ..., '4', '5', '6'], dtype=object)

In [4]:
knn_pred = knn_clf.predict(X_test)

In [5]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, knn_pred)

0.9688

In [7]:
grid_params = {
    'n_neighbors': [3,4,5],
    'weights': ['uniform', 'distance']
}

In [ ]:
gs = GridSearchCV(knn_clf, grid_params, cv=3)
gs.fit(X_train, y_train)
print('best parameters:', gs.best_params)
print('best score:', gs.best_score_)
print('best test score:', gs.score(X_test, y_test))

# 너무 오래 걸려서 실행 생략. 
# Best Params: {'n_neighbors':4, 'weights'='distance'}
# Best Score: 0.9703500000000002

In [17]:
knn_clf2 = KNeighborsClassifier(n_neighbors=4, weights='distance')
knn_clf2.fit(X_train, y_train)
accuracy_score(y_test, knn_clf2.predict(X_test))

0.9714

----------------------
2번

In [12]:
from scipy.ndimage.interpolation import shift

def shift_image(image, dx, dy):
  image = image.reshape((28,28))
  shifted_image = shift(image, [dy,dx], cval=0, mode='constant')
  return shifted_image.reshape([-1])

X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((1,0), (-1,0), (0,1), (0,-1)):
  for image, label in zip(X_train, y_train):
    X_train_augmented.append(shift_image(image, dx, dy))
    y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [15]:
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

knn_clf = KNeighborsClassifier(n_neighbors=4, weights='distance')

knn_clf.fit(X_train_augmented, y_train_augmented)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [16]:
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9763